# Example of `Onsager`/`VASP`/`??` workflow, including using `automator` to generate VASP input
This is an example notebook to showcase the workflow of `Onsager` in order to generate input files for `VASP`.

It then runs `VASP`, and then will run the `??` code that uses the `VASP` outputs to determine transport coefficients.

In [1]:
import sys
sys.path.extend(['../', './'])
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline
from onsager import crystal, OnsagerCalc
from parsl import *

### Step 1: Make a crystal
In this step, we make a crystal. This requires a "lattice constant", $a_0$, which is the size of the cell (units: nanometers), and the lattice vectors. In this example, we'll use one of the existing constructors in `crystal`, corresponding to a hexagonal-close packed (HCP) crystal. We'll name the chemistry "Mg", which is mainly for readability. This is not the exact lattice constant for Mg, though.

In [2]:
a0 = 3.21
Mgcrys = crystal.Crystal.HCP(a0, chemistry='Mg')
print(Mgcrys)

#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]


### Step 2: Identify the jump vectors for the vacancy
This requires us to

1. Identify the sublattice for diffusion. This is called the `chemistry`, which is an index. In HCP, there is only one sublattice, so `chemistry` = 0.
2. Find the jump vectors on that sublattice. This is callde the `jumpnetwork`. We just use a cutoff distance to search for all sublattice sites within a certain distance.

In [3]:
chemistry = 0
sitelist = Mgcrys.sitelist(chemistry)
print(sitelist)

[[0, 1]]


In [4]:
cutoff = 1.01*a0
jumpnetwork = Mgcrys.jumpnetwork(chemistry, cutoff)
for jn in jumpnetwork:
    print('symmetry related jumps:')
    for (i,j), dx in jn:
        print("  {}->{}, dx={}".format(i, j, dx))

symmetry related jumps:
  1->1, dx=[  3.21000000e+00   2.22044605e-16   0.00000000e+00]
  1->1, dx=[ -3.21000000e+00  -2.22044605e-16  -0.00000000e+00]
  1->1, dx=[ 1.605       2.77994155  0.        ]
  1->1, dx=[-1.605      -2.77994155 -0.        ]
  1->1, dx=[ 1.605      -2.77994155  0.        ]
  1->1, dx=[-1.605       2.77994155 -0.        ]
  0->0, dx=[ -3.21000000e+00   2.22044605e-16   0.00000000e+00]
  0->0, dx=[  3.21000000e+00  -2.22044605e-16  -0.00000000e+00]
  0->0, dx=[-1.605       2.77994155  0.        ]
  0->0, dx=[ 1.605      -2.77994155 -0.        ]
  0->0, dx=[ 1.605       2.77994155  0.        ]
  0->0, dx=[-1.605      -2.77994155 -0.        ]
symmetry related jumps:
  1->0, dx=[ 1.605      -0.92664718 -2.62095402]
  0->1, dx=[-1.605       0.92664718  2.62095402]
  1->0, dx=[ 0.          1.85329436 -2.62095402]
  0->1, dx=[-0.         -1.85329436  2.62095402]
  1->0, dx=[ 1.605      -0.92664718  2.62095402]
  0->1, dx=[-1.605       0.92664718 -2.62095402]
  1->0, dx

### Step 3: Construct a diffuser
We now make a "vacancy diffuser": this will allow us to compute the diffusivity of a solute in our crystal that diffuses via a vacancy-mediated mechanism. The diffuser takes in the inputs that we have defined above. The only additional parameter is the "range" of vacancy-solute interactions; it is defined in terms of how many "jumps" away we can have; typically only 1 or 2 is needed.

In [5]:
Nrange = 1
Mgdiffuser = OnsagerCalc.VacancyMediated(Mgcrys, chemistry, sitelist, jumpnetwork, Nrange)
print(Mgdiffuser)

Diffuser for atom 0 (Mg), Nthermo=1
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
vacancy configurations:
v:+0.333,+0.667,+0.250
solute configurations:
s:+0.333,+0.667,+0.250
solute-vacancy configurations:
s:+0.667,+0.333,+0.750-v:+0.333,+0.667,+0.250
s:+0.333,+0.667,+0.250-v:+0.333,-0.333,+0.250
omega0 jumps:
omega0:v:+0.667,+0.333,+0.750^v:+1.667,+1.333,+0.750
omega0:v:+0.667,+0.333,+0.750^v:+1.333,+0.667,+0.250
omega1 jumps:
omega1:s:+0.333,+0.667,+0.250-v:+0.667,+0.333,+0.750^v:+1.667,+1.333,+0.750
omega1:s:+0.333,+0.667,+0.250-v:-0.333,+0.333,-0.250^v:+0.667,+1.333,-0.250
omega1:s:+0.667,+0.333,+0.750-v:+0.667,+1.333,+0.750^v:+1.667,+2.333,+0.750
omega1:s:+0.333,+0.667,+0.250-v:+0.667,+1.333,-0.250^v:+1.667,+2.333,-0.250
omega1:s:+0.667,+0.333,+0.750-v:-0.333,+0.333,+0.750^v:

### Step 4: Construct supercells
A supercell is a repeated arrangement of the unit cell, where we then introduce our defects. For this case, we'll make a $4\times4\times3$ supercell, which is repeated 4 times in $\mathbf a_1$ direction, 4 times in the $\mathbf a_2$ direction, and then 3 times in the $\mathbf a_3$ direction. That makes for a cell that has $4\times4\times3\times2 = 96$ atoms (before we start introducing defects, such as a vacancy or a solute). Right now, the `makesupercells` command may raise warnings if the supercells are "too small"; these warnings can be safely ignored (and we should probably want to consider adding a flag to suppress such warnings).

In [6]:
Nsupercell = np.diag([4,4,3])
print(Nsupercell)

[[4 0 0]
 [0 4 0]
 [0 0 3]]


In [7]:
help(Mgdiffuser.makesupercells)

Help on method makesupercells in module onsager.OnsagerCalc:

makesupercells(super_n) method of onsager.OnsagerCalc.VacancyMediated instance
    Take in a supercell matrix, then generate all of the supercells needed to compute
    site energies and transitions (corresponding to the representatives).
    
    Note: the states are lone vacancy, lone solute, solute-vacancy complexes in
    our thermodynamic range. Note that there will be escape states are endpoints of
    some omega1 jumps. They are not relaxed, and have no pre-existing tag. They will
    only be output as a single endpoint of an NEB run; there may be symmetry equivalent
    duplicates, as we construct these supercells on an as needed basis.
    
    We've got a few classes of warnings (from most egregious to least) that can issued
    if the supercell is too small; the analysis will continue despite any warnings:
    
    1. Thermodynamic shell states map to different states in supercell
    2. Thermodynamic shell states

In [8]:
Mgsupercells = Mgdiffuser.makesupercells(Nsupercell)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Supercell:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
too small: escape endpoint has mapping error
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Supercell:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
too small: escape endpoint has multiplicity issue
  """Entry point for launching an IPython kernel.


There dictionary has 5 pieces to it:

1. `states`: these are the configurations that need to be computed first, and include the vacancy by itself, the solute by itself, and the solute-vacancy complexes. This is a dictionary where the key is the tag of the state, and the value is the supercell.
2. `transitions`: the transition states to be computed. This is also a dictionary; the key is the tag of the transition, and the value is a tuple of two supercells: the initial state and final state.
3. `transmapping`: this "maps" (which means "transform") a configuration in `states` to the initial or final state of a transition. So, it is a dictionary where the keys are the same keys as in `transitions`. The value is then a tuple, corresponding to the initial state and final state: `( (tag of initial state, group operation to transform), (tag of final state, group operation to transform) )`. The tags inside there correspond to a state, and then the group operation to transform from the relaxed supercell output of VASP into the input of the NEB calcuation.
4. `indices`: a dictionary, where the keys are *all* of the tags that are present (states, transition), and the value is tuple of `(type of configuration, number)`, such as `('vacancy', 0)` or `('omega1', 12)`. This can be used to simplify the names of the directories when running.
5. `reference`: a single supercell that is defect free.

In [9]:
print(Mgsupercells.keys())

dict_keys(['states', 'transitions', 'transmapping', 'indices', 'reference'])


Below is what a supercell looks like. When pretty-printed, it gives the underlying lattice, the supercell vectors, the chemistry in the cell (in this case, its all Mg, no solute), the "Kroger-Vink" notation, which tells you what defects are in the cell. In this case, it is empty as there are no defects. What follows are all of the atomic positions in the supercell, as well as the *ordering*: when the `POSCAR` (position file) is written for `VASP`, we will output the atomic positions in this specific order. Physically, the energy is independent of the ordering; however, when we do the transition states, the ordering does matter, because we specify how each atom moves from the initial to the final state, and that assumes that the first atom in the initial state maps to the first atom in the final state, and so on.

In [10]:
print(Mgsupercells['reference'])

Supercell of crystal:
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
Supercell vectors:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
Chemistry: Mg(96),solute(0)
Kroger-Vink: 
Positions:
[ 0.08333333  0.16666667  0.75      ] Mg
[ 0.16666667  0.08333333  0.91666667] Mg
[ 0.08333333  0.16666667  0.08333333] Mg
[ 0.16666667  0.08333333  0.25      ] Mg
[ 0.08333333  0.16666667  0.41666667] Mg
[ 0.16666667  0.08333333  0.58333333] Mg
[ 0.08333333  0.41666667  0.75      ] Mg
[ 0.16666667  0.33333333  0.91666667] Mg
[ 0.08333333  0.41666667  0.08333333] Mg
[ 0.16666667  0.33333333  0.25      ] Mg
[ 0.08333333  0.41666667  0.41666667] Mg
[ 0.16666667  0.33333333  0.58333333] Mg
[ 0.08333333  0.66666667  0.75      ] Mg
[ 0.16666667  0.58333333  0.91666667] Mg
[ 0.08333333  0.66666667  0.08333333] Mg
[ 0.1666

Now, the states. This is can be a very long list, and the supercells are all quite long, but you can now see examples of vacancies (Kroger-Vink notation: v_Mg), solutes (Kroger-Vink notation: solute_Mg).

In [11]:
print("Number of states:", len(Mgsupercells['states']))
print(Mgsupercells['states'].keys())
for k, v in Mgsupercells['states'].items():
    print('tag: {}'.format(k))
    print(v)

Number of states: 4
dict_keys(['v:+0.333,+0.667,+0.250', 's:+0.333,+0.667,+0.250', 's:+0.667,+0.333,+0.750-v:+0.333,+0.667,+0.250', 's:+0.333,+0.667,+0.250-v:+0.333,-0.333,+0.250'])
tag: v:+0.333,+0.667,+0.250
Supercell of crystal:
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
Supercell vectors:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
Chemistry: Mg(95),solute(0)
Kroger-Vink: v_Mg
Positions:
[ 0.08333333  0.16666667  0.75      ] Mg
[ 0.16666667  0.08333333  0.91666667] Mg
[ 0.08333333  0.16666667  0.08333333] v
[ 0.16666667  0.08333333  0.25      ] Mg
[ 0.08333333  0.16666667  0.41666667] Mg
[ 0.16666667  0.08333333  0.58333333] Mg
[ 0.08333333  0.41666667  0.75      ] Mg
[ 0.16666667  0.33333333  0.91666667] Mg
[ 0.08333333  0.41666667  0.08333333] Mg
[ 0.16666667  0.33333333  0.25      ] Mg


The transition states. Much longer.

In [12]:
print("Number of transition states:", len(Mgsupercells['transitions']))
print(Mgsupercells['transitions'].keys())
for k, v in Mgsupercells['transitions'].items():
    print('tag: {}'.format(k))
    print('  initial state:')
    print(v[0])
    print('  final state:')
    print(v[1])

Number of transition states: 17
dict_keys(['omega0:v:+0.667,+0.333,+0.750^v:+1.667,+1.333,+0.750', 'omega0:v:+0.667,+0.333,+0.750^v:+1.333,+0.667,+0.250', 'omega1:s:+0.333,+0.667,+0.250-v:+0.667,+0.333,+0.750^v:+1.667,+1.333,+0.750', 'omega1:s:+0.333,+0.667,+0.250-v:-0.333,+0.333,-0.250^v:+0.667,+1.333,-0.250', 'omega1:s:+0.667,+0.333,+0.750-v:+0.667,+1.333,+0.750^v:+1.667,+2.333,+0.750', 'omega1:s:+0.333,+0.667,+0.250-v:+0.667,+1.333,-0.250^v:+1.667,+2.333,-0.250', 'omega1:s:+0.667,+0.333,+0.750-v:-0.333,+0.333,+0.750^v:+0.667,+1.333,+0.750', 'omega1:s:+0.667,+0.333,+0.750-v:+0.667,-0.667,+0.750^v:+1.667,+0.333,+0.750', 'omega1:s:+0.667,+0.333,+0.750-v:+1.667,+0.333,+0.750^v:+2.667,+1.333,+0.750', 'omega1:s:+0.667,+0.333,+0.750-v:+1.667,+1.333,+0.750^v:+2.667,+2.333,+0.750', 'omega1:s:+0.333,+0.667,+0.250-v:+0.667,+0.333,+0.750^v:+1.333,+0.667,+0.250', 'omega1:s:+0.333,+0.667,+0.250-v:-0.333,+0.333,-0.250^v:+0.333,+0.667,-0.750', 'omega1:s:+0.667,+0.333,+0.750-v:+0.667,+1.333,+0.750^v

Supercell of crystal:
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
Supercell vectors:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
Chemistry: Mg(94),solute(1)
Kroger-Vink: solute_Mg+v_Mg
Positions:
[ 0.08333333  0.16666667  0.75      ] Mg
[ 0.16666667  0.08333333  0.91666667] Mg
[ 0.08333333  0.16666667  0.08333333] solute
[ 0.16666667  0.08333333  0.25      ] Mg
[ 0.08333333  0.16666667  0.41666667] Mg
[ 0.16666667  0.08333333  0.58333333] Mg
[ 0.08333333  0.41666667  0.75      ] Mg
[ 0.16666667  0.33333333  0.91666667] Mg
[ 0.08333333  0.41666667  0.08333333] Mg
[ 0.16666667  0.33333333  0.25      ] Mg
[ 0.08333333  0.41666667  0.41666667] Mg
[ 0.16666667  0.33333333  0.58333333] Mg
[ 0.08333333  0.66666667  0.75      ] Mg
[ 0.16666667  0.58333333  0.91666667] Mg
[ 0.08333333  0.66666667  0.083

  initial state:
Supercell of crystal:
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
Supercell vectors:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
Chemistry: Mg(94),solute(1)
Kroger-Vink: solute_Mg+v_Mg
Positions:
[ 0.08333333  0.16666667  0.75      ] Mg
[ 0.16666667  0.08333333  0.91666667] Mg
[ 0.08333333  0.16666667  0.08333333] Mg
[ 0.16666667  0.08333333  0.25      ] solute
[ 0.08333333  0.16666667  0.41666667] Mg
[ 0.16666667  0.08333333  0.58333333] Mg
[ 0.08333333  0.41666667  0.75      ] Mg
[ 0.16666667  0.33333333  0.91666667] Mg
[ 0.08333333  0.41666667  0.08333333] Mg
[ 0.16666667  0.33333333  0.25      ] Mg
[ 0.08333333  0.41666667  0.41666667] Mg
[ 0.16666667  0.33333333  0.58333333] Mg
[ 0.08333333  0.66666667  0.75      ] Mg
[ 0.16666667  0.58333333  0.91666667] Mg
[ 0.08333333  

[ 0.16666667  0.08333333  0.25      ] solute
  final state:
Supercell of crystal:
#Lattice:
  a1 = [ 1.605      -2.77994155  0.        ]
  a2 = [ 1.605       2.77994155  0.        ]
  a3 = [ 0.          0.          5.24190805]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]
Supercell vectors:
[[4 0 0]
 [0 4 0]
 [0 0 3]]
Chemistry: Mg(94),solute(1)
Kroger-Vink: solute_Mg+v_Mg
Positions:
[ 0.08333333  0.16666667  0.75      ] Mg
[ 0.16666667  0.08333333  0.91666667] Mg
[ 0.08333333  0.16666667  0.08333333] Mg
[ 0.16666667  0.08333333  0.25      ] solute
[ 0.08333333  0.16666667  0.41666667] Mg
[ 0.16666667  0.08333333  0.58333333] Mg
[ 0.08333333  0.41666667  0.75      ] Mg
[ 0.16666667  0.33333333  0.91666667] Mg
[ 0.08333333  0.41666667  0.08333333] Mg
[ 0.16666667  0.33333333  0.25      ] Mg
[ 0.08333333  0.41666667  0.41666667] Mg
[ 0.16666667  0.33333333  0.58333333] Mg
[ 0.08333333  0.66666667  0.75      ] Mg
[ 0.16666667

### Step 5: Use Automator to convert the supercells into VASP input
We use the Automator module to generate `VASP` input.

In [13]:
from onsager import automator
import tarfile
help(automator.supercelltar)

Help on function supercelltar in module onsager.automator:

supercelltar(tar, superdict, filemode=436, directmode=509, timestamp=None, INCARrelax='SYSTEM = {system}\nPREC = High\nISIF = 2\nEDIFF = 1E-8\nEDIFFG = -10E-3\nIBRION = 2\nNSW = 50\nISMEAR = 1\nSIGMA = 0.1\n# ENCUT =\n# NGX =\n# NGY =\n# NGZ =\n# NGXF =\n# NGYF =\n# NGZF =\n# NPAR =\nLWAVE  = .FALSE.\nLCHARG = .FALSE.\nLREAL  = .FALSE.\nVOSKOWN = 1\n', INCARNEB='SYSTEM = {system}\nPREC = High\nISIF = 2\nEDIFF = 1E-8\nEDIFFG = -10E-3\nIBRION = 2\nNSW = 50\nISMEAR = 1\nSIGMA = 0.1\n# ENCUT =\n# NGX =\n# NGY =\n# NGZ =\n# NGXF =\n# NGYF =\n# NGZF =\n# NPAR =\nLWAVE  = .FALSE.\nLCHARG = .FALSE.\nLREAL  = .FALSE.\nVOSKOWN = 1\nIMAGES = 1\nSPRING = -5\nLCLIMB = .TRUE.\nNELMIN = 4\nNFREE = 10\n', KPOINTS='Gamma\n1\nReciprocal\n0. 0. 0. 1.\n', basedir='', statename='relax.', transitionname='neb.', IDformat='{:02d}', JSONdict='tags.json', YAMLdef='supercell.yaml')
    Takes in a tarfile (needs to be open for writing) and a supercelldic

In [14]:
#with tarfile.open('example-supercells.tar.gz', mode='w:gz') as tar:
#    automator.supercelltar(tar, Mgsupercells)

Create uncompressed tar file with VASP inputs

(TBD: It would be better to write the directories themselves, but that would involve understanding the `supercelltar()` method and modifying it, while this way just works with the existing code)

In [15]:
chemistry_name='example'
dir_name=chemistry_name+'-supercells'
tar_name=dir_name+'.tar'
statename='relax.'
transitionname='neb.'
digits_in_file_extensions=3
IDformat='{:0'+str(digits_in_file_extensions)+'d}'

with tarfile.open(tar_name, mode='w') as tar:
    automator.supercelltar(tar, Mgsupercells, statename=statename, transitionname=transitionname, IDformat=IDformat)

now take the files out of the tar file - this will create a directory called `example-supercells` that contains the inputs needed to run VASP

(TBD: what about errors or exceptions?)

In [16]:
with tarfile.open(tar_name, mode='r') as tar:
    tar.extractall(path=dir_name)

### Step 6: Get ready to run VASP
Now we need to run VASP - we need to run it using the inputs in the relax.?? directories, then we need to run make, then we need to run VASP again using the inputs in the neb.?? directories.


In [17]:
print("Number of", statename+"* directories:", len(Mgsupercells['states']))
print("Number of", transitionname+"* directories:", len(Mgsupercells['transitions']))


Number of relax.* directories: 4
Number of neb.* directories: 17


A fake `VASP` run might just `cp POSCAR CONTCAR`.

need to build POTCAR input to VASP first, based on chemisty and solute, taken from VASP-supplied files.  Dallas suggest Al as a good quick first solute.

need to `zcat potUSPP_LDA/Mg/POTCAR.Z potUSPP_LDA/Al/POTCAR.Z > POTCAR`

really, built Parsl bash app that does this